In [1]:
from osgeo import gdal, ogr, osr
from sklearn.ensemble import RandomForestClassifier
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [3]:
a = gdal.Open("./clipped/2011.tif")

'./clipped/2011.tif'

In [7]:
folder_path = "./rad-corr/"
flist = os.listdir(folder_path)
for i in range(0, len(flist), 2):
    print(flist[i])
    if flist[i][-4:] != ".tif":
        continue
    half = gdal.Open(folder_path + flist[i+1]) # 500m分辨率波段
    quater = gdal.Open(folder_path + flist[i]) # 250m
    red_band = quater.GetRasterBand(1).ReadAsArray() # 红光波段
    nir_band = quater.GetRasterBand(2).ReadAsArray()
    green_band = half.GetRasterBand(2).ReadAsArray()
    blue_band = half.GetRasterBand(1).ReadAsArray()
    swir_band = half.GetRasterBand(3).ReadAsArray() # 短波红外, 该band的SNR较低

    x,y=nir_band.shape
    swir_band = cv2.resize(swir_band, dsize=(y,x),
                            interpolation=cv2.INTER_LINEAR)
    blue_band = cv2.resize(blue_band, dsize=(y,x),
                            interpolation=cv2.INTER_LINEAR)
    green_band = cv2.resize(green_band, dsize=(y,x),
                            interpolation=cv2.INTER_LINEAR)
    
    ci = (nir_band + 2 * swir_band) / (red_band + green_band + blue_band)
    ci = np.nan_to_num(ci, 1)
    # https://blog.csdn.net/ssshyeong/article/details/125947734
    # 云检测 
    ci[np.fabs(ci - 1) < 1] = 1
    ci[np.fabs(ci - 1) >= 1] = 0
    out_ds = gdal.GetDriverByName("GTiff").Create(
        "./cloud_mask/" + flist[i][:-6]+"cloud-mask.tif", 
        quater.RasterXSize, quater.RasterYSize, 1, gdal.GDT_Int64)
    
    out_ds.WriteArray(ci)
    out_ds.SetGeoTransform(quater.GetGeoTransform())
    out_ds.SetProjection(quater.GetProjection())
    out_ds.FlushCache()
    out_ds = None

MOD021KM.A2011181.0325.061.2017324101641_5.tif


C:\Users\yhw60\AppData\Local\Temp\ipykernel_14348\77443282.py:23: RuntimeWarning: invalid value encountered in divide
  ci = (nir_band + 2 * swir_band) / (red_band + green_band + blue_band)


MOD021KM.A2012182.0200.061.2017334143411_5.tif
MOD021KM.A2012182.0335.061.2017334143816_5.tif
MOD021KM.A2013181.0305.061.2017299063350_5.tif
MOD021KM.A2014181.0235.061.2017312140235_5.tif
MOD021KM.A2015181.0205.061.2017321044912_5.tif
MOD021KM.A2015181.0345.061.2017321045213_5.tif
MOD021KM.A2016182.0220.061.2017326170448_5.tif
MOD021KM.A2017181.0325.061.2017315065028_5.tif
MOD021KM.A2018181.0255.061.2018181133528_5.tif
MOD021KM.A2019181.0225.061.2019181131027_5.tif


In [39]:
#nir_band.mean(), swir_band.mean(), green_band.shape, red_band.shape, blue_band.shape
ci.max(), ci.min(), ci.mean()

(1.0, 0.0, 0.6368364772710532)

In [11]:
# 去云
folder_path = "./rad-corr/"
cloud_path = "./cloud_mask/"
out_path = "./cloud-clip/"
cloud_list = os.listdir(cloud_path)
flist = os.listdir(folder_path)
# i = 0
for i in range(0, len(flist), 2):
    if flist[i][-4:] != ".tif":
        continue
    quater = gdal.Open(folder_path + flist[i]) # 250m  
    half = gdal.Open(folder_path + flist[i+1]) # 500m
    cloud = gdal.Open(cloud_path + cloud_list[i % 2]) # 云掩膜
    cloud_band = cloud.GetRasterBand(1).ReadAsArray()
    out_ds = gdal.GetDriverByName("GTiff").Create(
        out_path + flist[i][:-4] + "cloud-clip22.tif", 
        quater.RasterXSize, quater.RasterYSize, (2+5)*2, gdal.GDT_Int64)
    # 读取每一个波段的数据
    band_list = []
    band_list.append(quater.GetRasterBand(1).ReadAsArray())
    band_list.append(quater.GetRasterBand(2).ReadAsArray())
    band_list.append(quater.GetRasterBand(3).ReadAsArray())
    band_list.append(quater.GetRasterBand(4).ReadAsArray())
    band_list.append(half.GetRasterBand(1).ReadAsArray())
    band_list.append(half.GetRasterBand(2).ReadAsArray())
    band_list.append(half.GetRasterBand(3).ReadAsArray())
    band_list.append(half.GetRasterBand(4).ReadAsArray())
    band_list.append(half.GetRasterBand(5).ReadAsArray())
    band_list.append(half.GetRasterBand(6).ReadAsArray())
    band_list.append(half.GetRasterBand(7).ReadAsArray())
    band_list.append(half.GetRasterBand(8).ReadAsArray())
    band_list.append(half.GetRasterBand(9).ReadAsArray())
    band_list.append(half.GetRasterBand(10).ReadAsArray())
    j = 0
    for band in band_list:
        #out_band = out_ds.GetRasterBand(j+1).ReadAsArray()
        out_band = band
        x, y = band.shape
        resize_cloud = cv2.resize(cloud_band+0.0001, dsize=(y,x), 
                                  interpolation=cv2.INTER_LINEAR)
        out_band[resize_cloud == 0.0] = 255
        # out_band[cloud_band != 1.0] = band
        #print(out_band.max(), out_band.min(), out_band.mean())
        out2 = out_ds.GetRasterBand(j+1)
        out2.WriteArray(out_band)
        out3 = out_ds.GetRasterBand(j+1)
        kkkkk = out3.ReadAsArray()
        print(kkkkk.mean())
        j+=1
        #for k in range(row):
        #    for m in range(col):
        #        if cloud_band[k][m] == 1.0:
        #            out_band[k][m] = 65535.0
        #        else:
        #            out_band[k][m] = band[k][m]
        #pass
    out_ds.SetProjection(quater.GetProjection())
    out_ds.SetGeoTransform(quater.GetGeoTransform())
    out_ds.FlushCache()
    out_ds = None
    print(out_path + flist[i][:-4] + "cloud-clip.tif" + " complete clip!")
    pass


141.74538557694964
112.46171026823946
290.61047957125885
373.3084940191718
208.22363583938784
171.0403548785878
43.906658678416534
14.501920633166662
3.137978086411227
328.3979096452941
301.85805650708585
304.9739884043758
198.88934570015746
114.7792787784193
./cloud-clip/MOD021KM.A2011181.0325.061.2017324101641_5cloud-clip.tif complete clip!
62.00938546825179
42.20126842277892
127.0870440569455
139.94620252824242
113.45557509181144
80.14953014408437
16.064506207690116
5.154228344777278
1.1729627180129656
179.00974877091858
141.4394847446934
111.42341518845318
70.81865622821215
43.947181850280586
./cloud-clip/MOD021KM.A2012182.0200.061.2017334143411_5cloud-clip.tif complete clip!
121.8277069122014
101.9803999577009
249.89002686180487
338.6408286000668
171.84620558457502
144.9744782394701
41.81070776490009
15.022370258669637
3.570326241985546
271.0907406357393
255.92335974215666
290.5562697564381
206.3032008456072
130.63172070345846
./cloud-clip/MOD021KM.A2012182.0335.061.2017334143816_